# Introduction to Data Science
## Lab 6: Test for importance of a subset of predictors

In this problem, we want to investigate another data set.
The data come from a study of [Stamey et al. (1989)](https://www.sciencedirect.com/science/article/pii/S002253471741175X?via%3Dihub).
It consists of some clinical measurements of 97 patients, who where about to receive a prostatectomy.

The predictors are:
* lcavol - Logarithm of cancer volume
* lweight - Logarithm of prostate weight
* age - Age of patient
* lbph - Logarithm of amount of benign prostatic hyperplasia
* svi - Seminal vesicle invasion
* lcp - Logarithm of capsular penetration
* gleason - Gleason score
* pgg45 - Percent of Gleason scores 4 or 5

The variable that we want to predict is:
* lpsa - Level of prostate-specific antigen

There are no missing values.

The first task is to download and import the `prostate` data set.
You can find it [here](https://bildungsportal.sachsen.de/opal/auth/RepositoryEntry/26393509888/CourseNode/102441889726288/ps6).

Taking a short look at the data reveals that the data is seperated by tabs.
You can use the convenient `pandas` method `read_csv` with the options `sep = "\t"` and `index_col = 0`.

In [1]:
import pandas as pd

# Import prostate cancer dataset
pc = pd.read_csv('prostate.data', sep="\t", index_col=0)

The method `head` of a `pd.DataFrame` prints by default the first 5 rows of the `DataFrame`, which is suitable for getting an overview of the data.

**Task**: Try it!

In [2]:
# YOUR CODE HERE
pc.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


As you can see, there is a column named **train**, which is either `"T"` (True) or `"F"` (False).
This means that the data has already been split up into a training and a test set, which we will use in some minutes.

**Task**: You should alse take a look at the correlation matrix using the method `corr`.
What do you observe, especially concerning the correlation between the features?

In [3]:
# YOUR CODE HERE
corr = pc.corr()
corr.style.background_gradient(cmap='coolwarm')

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
lcavol,1.000000,0.280521,0.225000,0.027350,0.538845,0.675310,0.432417,0.433652,0.734460
lweight,0.280521,1.000000,0.347969,0.442264,0.155385,0.164537,0.056882,0.107354,0.433319
age,0.225000,0.347969,1.000000,0.350186,0.117658,0.127668,0.268892,0.276112,0.169593
lbph,0.027350,0.442264,0.350186,1.000000,-0.085843,-0.006999,0.077820,0.078460,0.179809
svi,0.538845,0.155385,0.117658,-0.085843,1.000000,0.673111,0.320412,0.457648,0.566218
lcp,0.675310,0.164537,0.127668,-0.006999,0.673111,1.000000,0.514830,0.631528,0.548813
gleason,0.432417,0.056882,0.268892,0.077820,0.320412,0.514830,1.000000,0.751905,0.368987
pgg45,0.433652,0.107354,0.276112,0.078460,0.457648,0.631528,0.751905,1.000000,0.422316
lpsa,0.734460,0.433319,0.169593,0.179809,0.566218,0.548813,0.368987,0.422316,1.000000


You should observe that there are many predictor variables that have a high correlation.
High correlation is always an *indicator* of multiple features having similar effects on the independent variable.

From now on, we concentrate on the numerical values of the data set:
- extract the predictors of the dataset as a `numpy.array` $X$ using the method `values` of a `pd.DataFrame`
- extract the column **lspa** as a `numpy.array` $y$

In [4]:
# YOUR CODE HERE
import numpy as np


predictor = pc.keys()[:-2] #OR predictor = ["lcavol","lweight","age","lbph","svi","lcp","gleason","pgg45"]


#First Solution
X=np.array(pc[predictor])
y=np.array(pc["lpsa"])

# OR by iloc
# X=pc.iloc[:,0:8].values
# y=pc.iloc[:,8].values

# wrong solution
# X1=pc.values[:,:-2]
# y1=pc.values[:,-2]
# I used X1 and y1 but it makes error in multipleTTest function. They have difference types.
# type(X[0,0])==type(X1[0,0])

In [5]:
import numpy as np

assert np.abs(X.mean() - 12.514554639313143) < 1e-10
assert np.abs(y.mean() - 2.4783868783505154) < 1e-10
assert X.shape == (97,8)
assert y.shape == (97,)

Before we further analyze our data, we want to normalize the predictor variables.
This technique is often necessary when comparing different kinds of inputs.

Here, we want to normalize the **predictor variables** such that the *normalized predictor variables* have mean $0$ and variance $1$.

**Caution**: There are a lot of functions that sound like they're doing the right thing, but there are a lot more ways to normalize a data set, e.g. $l^2$-normalization etc.

**Task**: Normalize the predictor variables, you can use the function `scale` from `scikit-learn`:

        from sklearn.preprocessing import scale

You should name the normalized variable `Xnorm`, otherwise the following code might not work.

In [6]:
# YOUR CODE HERE
from sklearn.preprocessing import scale
Xnorm = scale(X)

In [7]:
assert max(abs(Xnorm.mean(axis=0))) < 1e-10
assert max(abs(Xnorm.var(axis=0)-1)) < 1e-10

The next step is to devide the data set into a training and a test data set.
We can extract the column **train** by

    train = (pc["train"]=="T")
    
which is simply a `pandas Series` object containing the values `True` and `False`.
The nice thing about this object is that we can use this *filter* to get our training data by

    Xtrain = Xnorm[train,:]
    
The same indexing can be applied to $y$.

**Task**: Extract the training samples and save the new variables under `Xtrain` and `ytrain`.

In [8]:
# YOUR CODE HERE
train = pc["train"]=="T"
Xtrain = Xnorm[train,:]
ytrain = y[train]

In [9]:
assert abs(Xtrain[-1,-1] - 1.9822513682855871) < 1e-10
assert abs(Xtrain.mean() - 0.011451538411465602) < 1e-10
assert Xtrain.shape == (67,8)

assert abs(ytrain[50] - 3.3928290999999997) < 1e-10
assert abs(ytrain.mean() - 2.4523450850746267) < 1e-10
assert ytrain.shape == (67,)

Now, we want to fit a linear regression model on our **training set** using all of the predictor variables.
In the last tutorials, we have employed `numpy`s methods `polyfit` and `polyval`.
Here, you should use the following class:

    from sklearn.linear_model import LinearRegression
    
Have a look at its documentation and fit the model.
Store the trained intercept in the variable `intercept` as well as the regression coefficients (as a list) in the variable `coeffs`.

In [10]:
# YOUR CODE HERE
from sklearn.linear_model import LinearRegression
LR = LinearRegression()

reg = LR.fit(Xtrain,ytrain)
intercept = reg.intercept_
coeffs = reg.coef_

In [11]:
assert abs(intercept - 2.464932922123745) < 1e-10
assert abs(np.mean(coeffs) - 0.15837992642962145) < 1e-10

In OPAL, you find the file `multipleTTest.py`.
After downloading the file, you can import it using
    
    from multipleTTest import multipleTTest

This function enhances the capability of our previous method `computeTStatistic` slightly:
* You may now also provide a `labels` object that contains the names of the predictor variables and modifies only the output table. You can get the labels of a `pandas DataFrame` by
    
        labels = pc.keys()
     
* Additionally, the function `multipleTTest` returns the residual sum of squares of the linear regression fit. 
* You may set the optional input variable `includeIntercept` to `True`. This appends the intercept in the estimate, and you have not to put a column containing only ones by yourself.

**Task**: Execute the function for this data set. Store the RSS value as `RSS1`. Store also the number of variables in this model as `p1` (count the intercept as one variable!).
How many variables are not significant at a threshold of 5 %?
Store your answer in the variable 'notSign1'.

In [12]:
# YOUR CODE HERE
from multipleTTest import multipleTTest

p1=len(predictor)+1

RSS1=multipleTTest(Xtrain ,ytrain , True, predictor)


notSign = ['age', 'lcp', 'gleason', 'pgg45']
notSign1=len(notSign)

print (f'RSS = {RSS1}')
print (f"For {notSign} the p-value > 0.05, so they are not significate")

|  Coefficient | Estimate |    SE    | t-statistic |  p-value  |
---------------------------------------------------------------
|  Intercept   |   2.465  |  0.0893  |    27.60    | < 0.0001  |
|     lcavol   |   0.676  |  0.1260  |     5.37    | < 0.0001  |
|    lweight   |   0.262  |  0.0951  |     2.75    |   0.0079  |
|        age   |  -0.141  |  0.1008  |    -1.40    |   0.1681  |
|       lbph   |   0.209  |  0.1017  |     2.06    |   0.0443  |
|        svi   |   0.304  |  0.1230  |     2.47    |   0.0165  |
|        lcp   |  -0.287  |  0.1537  |    -1.87    |   0.0670  |
|    gleason   |  -0.021  |  0.1445  |    -0.15    |   0.8839  |
|      pgg45   |   0.266  |  0.1528  |     1.74    |   0.0875  |
RSS = 29.426384459908395
For ['age', 'lcp', 'gleason', 'pgg45'] the p-value > 0.05, so they are not significate


In [13]:
assert p1 == 9
assert abs(RSS1 - 29.4263844599084) < 1e-10
assert 'notSign1' in locals()


We want to compare the fit containing all predictor variables against a model containing only those variables that are significant at the $5~\%$ level. The variables that we exclude should be `age`, `lcp`, `gleason` and `pgg45`.

**Task**: Use the function `multipleTTest` to perform a test using only the significant variables.
Store the residual sum of squares as `RSS0` and the number of variables as `p0`.

In [14]:
# YOUR CODE HERE
from multipleTTest import multipleTTest

sing = list(set(predictor)-set(notSign))

X0 = pc[sing]
X0norm = scale(X0)
X0train = X0norm[train,:]
p0=len(sing)+1

RSS0=multipleTTest(X0train ,ytrain , True, sing)

|  Coefficient | Estimate |    SE    | t-statistic |  p-value  |
---------------------------------------------------------------
|  Intercept   |   2.471  |  0.0890  |    27.77    | < 0.0001  |
|     lcavol   |   0.593  |  0.1085  |     5.46    | < 0.0001  |
|    lweight   |   0.230  |  0.0941  |     2.44    |   0.0175  |
|        svi   |   0.277  |  0.1125  |     2.46    |   0.0167  |
|       lbph   |   0.202  |  0.1016  |     1.99    |   0.0512  |


In [15]:
assert p0 == 5


According to the lecture, 03_IntroDS_Regression.pdf slide 34, we may now test the reduced model against the full model using an appropriate $F$-statistic.
Thus, we test the null hypothesis is:

$$ \textbf{H}_0: \beta_{\text{age}} = \beta_{\text{lcp}} = \beta_{\text{gleason}} = \beta_{\text{pgg45}} = 0$$

against

$$ \textbf{H}_1: \text{at least one of the variables } \beta_{\text{age}}, \beta_{\text{lcp}}, \beta_{\text{gleason}}, \beta_{\text{pgg45}} \text{ is not zero } $$

We can do this by computing the $F$-stastitic:

$$ F = \frac{(RSS_0 - RSS_1) / (p_1 - p_0)}{RSS_1 / (n - p_1)} $$

while $n$ is the number of training samples.

According to our assumptions, $F$ will have an $F$-distribution with $(p_1-p_0, n-p_1)$ degrees of freedom.

**Task**: Compute the value of the test statistic and store it in a variable `F`.

In [16]:
# YOUR CODE HERE
df1=p1-p0
df2=len(ytrain)-p1
F=((RSS0-RSS1)/df1)/(RSS1/df2)
F

1.669754884637527

In [17]:
assert 'F' in locals()


To compute the $p$-value, we may import the $F$-distribution by

    from scipy.stats import f
    
**Task**: Determine the corresponding $p$-value and store the value in a variable `pval`.
Assuming a level of significance of 5%, can we safely reject the null hypothesis? Store either `True` or `False` in the variable `reject_null`.

In [18]:
# YOUR CODE HERE
from scipy.stats import f
pval = 1-f.cdf(F,df1,df2)
reject_null = (pval<0.05)

print(f'pval={pval} and reject null = {reject_null}')

pval=0.16933707265225018 and reject null = False


In [19]:
assert 'pval' in locals()
assert 'reject_null' in locals()
